In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(words[:5])

['emma', 'olivia', 'ava', 'isabella', 'sophia']


In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [4]:
# build the dataset
block_size = 5 # context length

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42442)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182646, 5]) torch.Size([182646])
torch.Size([22789, 5]) torch.Size([22789])
torch.Size([22711, 5]) torch.Size([22711])


In [5]:
# utility function to compare manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10 
n_hidden = 64

g = torch.Generator().manual_seed(42442)
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 #useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

5417


In [7]:
batch_size = 32
n = batch_size # convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [8]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time
emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# activation
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.5830, grad_fn=<NegBackward0>)

In [9]:
# Backprop through the whole thing manually

# only the values indexed at logprobs[range(n), Yb] affects the loss. these are averaged over the batch
# gradient of each value in an average is -1/n. -> set to 1/n for each
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n
cmp('logprobs', dlogprobs, logprobs)

# log is applied to probs to get logprobs. torch uses ln
# derivative of ln(x) is 1/x
dprobs = dlogprobs * probs**-1
cmp('probs', dprobs, probs)

# shape of counts_sum_inv is 32,1 so it is broadcast across counts
# so when counts_sum_inv[0] increases by 1, 27 params of counts[0] change
# sum those 27 params to get the total gradient
dcounts_sum_inv = (dprobs * counts).sum(1, keepdim=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# dy/dx of x**-1 is -1*x**-2
dcounts_sum = dcounts_sum_inv * -1*counts_sum**-2
cmp('counts_sum', dcounts_sum, counts_sum)

# counts used twice
dcounts = dprobs * counts_sum_inv + dcounts_sum
cmp('counts', dcounts, counts)

dnorm_logits = dcounts * norm_logits.exp()
cmp('norm_logits', dnorm_logits, norm_logits)

# subtraction so -1. But broadcast so must sum along the broadcast dimension
dlogit_maxes = (dnorm_logits * -torch.ones_like(logit_maxes)).sum(1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

# dnorm_logits is jsut *1. + get indices of max of each logit row and one hot them to 27 wide rows
# ie find the indices that affect the gradient. These are 1 and the rest are 0 hence one_hot
dlogits = (dnorm_logits) + (dlogit_maxes * F.one_hot(logits.max(1).indices, logits.shape[1]))
cmp('logits', dlogits, logits)

# h value affects all values in same row of logits
# logits[1,1] = h[1,1]*W2[1,1] + h[1,2]*W2[2,1] + ... + h[1,64]*W2[64,1] + b2[1]
# logits[1,2] = h[1,1]*W2[1,2] + h[1,2]*W2[2,2] + ... + h[1,64]*W2[64,2] + b2[2]
# ...
# logits[1,27] = h[1,1]*W2[1,27] + h[1,2]*W2[2,27] + ... + h[1,64]*W2[64,27] + b2[27]
# dL/dh[1,1] is the sum of all the affects of h[1,1]
# dL/dh[1,1] = dL/dlogits[1,1]*W2[1,1] + dL/dlogits[1,2]*W2[1,2] + ... + dL/dlogits[1,27]*W2[1,27]
# the above is the equivalent to matrix multiplication of dL/dlogits[1,:] and W2.T[:,1]
dh = dlogits @ W2.T
cmp('h', dh, h)

# same as before except using h and multiplying on the left
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

# each b2 has a gradient of 1 but for each entire column ie 1*values of each column of dlogits
# sum the columns to get the total gradient
db2 = dlogits.sum(0, keepdim=True)
cmp('b2', db2, b2)

dhpreact = dh * (1 - h**2)
cmp('hpreact', dhpreact, hpreact)

dbngain = (dhpreact * bnraw).sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)

dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)

dbnraw = (bngain * dhpreact)
cmp('bnraw', dbnraw, bnraw)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

#bnvar_inv = (bnvar + 1e-5)**-0.5 -> apply exponent rule
dbnvar = (-0.5*((bnvar + 1e-5)**-1.5))  * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
dbndiff2 = dbnvar * (1.0/(n-1))
#dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

dbndiff = dbndiff2 * (2.0 * bndiff) + (bnvar_inv * dbnraw)
cmp('bndiff', dbndiff, bndiff)

dbnmeani = -dbndiff.sum(0, keepdim=True)
cmp('bnmeani', dbnmeani, bnmeani)

# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
dhprebn = (dbndiff * torch.ones_like(hprebn)) + (dbnmeani * torch.ones_like(hprebn) * 1.0/(n))
cmp('hprebn', dhprebn, hprebn)

# hprebn = embcat @ W1 + b1 # hidden layer pre-activation
dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)

dW1 = embcat.T @ dhprebn
cmp('W1', dW1, W1)

db1 = dhprebn.sum(0, keepdim=True)
cmp('b1', db1, b1)

demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)

dC = torch.zeros_like(C)
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[i,j]
        dC[ix] += demb[i,j]
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [10]:
# Backprop through cross_entropy but all in one go

loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.583031415939331 diff: -2.384185791015625e-07


In [11]:
# backward pass

dlogits = F.softmax(logits, dim=1)
dlogits[range(n), Yb] -= 1.0
dlogits /= n

cmp('logits', dlogits, logits)

logits          | exact: False | approximate: True  | maxdiff: 1.0477378964424133e-08


In [12]:
# backprop through batchnorm but all in one go

hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [13]:
# backward pass

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0, keepdim=True) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0, keepdim=True))

cmp('hprebn', dhprebn, hprebn) # approx true

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [14]:
# Train with manual backward pass
with torch.no_grad():
  n_embd = 10
  n_hidden = 400
  vocab_size = 27

  g = torch.Generator().manual_seed(42442)
  C  = torch.randn((vocab_size, n_embd),            generator=g)
  # Layer 1
  W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)

  # Layer 2
  W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.01
  b2 = torch.randn(vocab_size,                      generator=g) * 0

  bngain = torch.ones((1, n_hidden))
  bnbias = torch.zeros((1, n_hidden))

  bn_mean_running = torch.zeros((1, n_hidden))
  bn_var_running = torch.ones((1, n_hidden))

  parameters = [C, W1, W2, b2, bngain, bnbias]
  print(sum(p.nelement() for p in parameters)) # number of parameters in total

  # same optimization as last time
  max_steps = 200000
  batch_size = 32
  momentum = 0.01
  n = batch_size # convenience
  lossi = []

  for i in range(max_steps):
    # minibatch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)

    # forward pass
    emb = C[Xtr[ix]]
    embcat = emb.view(emb.shape[0], -1)
    # Linear layer
    # -------------------------------------------------------------
    hprebn = embcat @ W1
    # Batch Normalisation layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
      p.grad = None

    # equivalent to loss.backward()
    dlogits = F.softmax(logits, dim=1)
    dlogits[range(n), Ytr[ix]] -= 1.0
    dlogits /= n
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    dhpreact = dh * (1.0 - h**2)
    bn_mean_running = (1-momentum) * bn_mean_running + momentum * bnmean
    bn_var_running = (1-momentum) * bn_var_running + momentum * bnvar

    dbngain = (dhpreact * bnraw).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    Xb = Xtr[ix]
    for j in range(Xb.shape[0]):
        for k in range(Xb.shape[1]):
            ix = Xb[j,k]
            dC[ix] += demb[j,k]
    grads = [dC, dW1, dW2, db2, dbngain, dbnbias]
    
    # update params
    lr = 0.01 if i < 100000 else 0.005
    for p, grad in zip(parameters, grads):
      p.data += -lr * grad

    if i % 10000 == 0:
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
      lossi.append(loss.log10().item())

31897
      0/ 200000: 3.3503
  10000/ 200000: 2.1159
  20000/ 200000: 2.6357
  30000/ 200000: 2.1498
  40000/ 200000: 2.3001
  50000/ 200000: 2.1886
  60000/ 200000: 1.8413
  70000/ 200000: 2.1714
  80000/ 200000: 2.6130
  90000/ 200000: 2.3408
 100000/ 200000: 1.8101
 110000/ 200000: 2.0721
 120000/ 200000: 2.3392
 130000/ 200000: 2.0538
 140000/ 200000: 2.0212
 150000/ 200000: 2.2308
 160000/ 200000: 2.3330
 170000/ 200000: 2.2257
 180000/ 200000: 1.9566
 190000/ 200000: 2.0826


In [15]:
# evaluate train and val loss

@torch.no_grad()
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x]
  hpreact = emb.view(-1, block_size*n_embd) @ W1 # bias does nothing now as it is negated by the normalisation - removed
  hpreact = bngain * (hpreact - bn_mean_running) / (bn_var_running + 1e-5)**0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0439820289611816
val 2.102834939956665


In [21]:
# sample from the model
g = torch.Generator().manual_seed(42442)

for _ in range(10):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

ise.
bryngela.
kurace.
raelin.
marce.
piarleony.
reenara.
rile.
shrann.
karedly.
